In [3]:
pip install opencv-python


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.3/39.5 MB 9.5 MB/s eta 0:00:05
   --- ------------------------------------ 3.4/39.5 MB 10.6 MB/s eta 0:00:04
   --- ------------------------------------ 3.4/39.5 MB 10.6 MB/s eta 0:00:04
   --- ------------------------------------ 3.4/39.5 MB 10.6 MB/s eta 0:00:04
   --- ------------------------------------ 3.4/39.5 MB 10.6 MB/s eta 0:00:04
   --- ------------------------------------ 3.4/39.5 MB 10.6 MB/s eta 0:00:04
   --- ------------------------------------ 3.7/39.5 MB 2.7 MB/s eta 0:00:14
   ----- ---------------------------------- 5.5/39.5 MB 3.3 MB/s eta 0:00:11
   ------- -------------------------------- 7.3/39.5 MB 4.1 MB/s eta 0:00:08
   ------- -------------------------------- 7.3/39.5 MB 4.1 MB/s eta 0:00:08
   ------- -------------------------------- 7.3/39.5 MB 4.1 MB/s eta 0:00:08
   --------- ------------------------------ 9.2/39.5 MB 3.7 MB/s eta 0:00:09
 

In [1]:
import cv2
print(cv2.__version__)


4.11.0


In [7]:
pip install mediapipe --user


  Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.5.0-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.5.0-cp312-cp312-win_amd64.whl.metadata (1.0 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl (51.0 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached jax-0.5.0-py3-none-any.whl (2.3 MB)
Using cached jaxlib-0.5.0-cp312-cp312-win_amd64.whl (63.9 MB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)
Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl (210 kB)
Note: you may need to restart the kernel to use updated

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

In [11]:
# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils


In [9]:
# # Create a canvas
# canvas = np.zeros((480, 640, 3), dtype="uint8")  # Black canvas

# # Start video capture
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Flip the frame
#     frame = cv2.flip(frame, 1)

#     # Draw something on the canvas (e.g., a rectangle and text)
#     cv2.rectangle(canvas, (100, 100), (300, 300), (0, 255, 0), -1)  # Green square
#     cv2.putText(canvas, "Canvas Overlay", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

#     # Combine the frame and canvas
#     combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

#     # Display the combined frame
#     cv2.imshow("Combined", combined)

#     # Exit on pressing 'q'
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [23]:
# Create a canvas
canvas = np.zeros((480, 640, 3), dtype="uint8")  # Black canvas

# Start video capture
cap = cv2.VideoCapture(0)

# Check if the webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Set webcam resolution to 640x480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# Flag to track whether drawing is happening
drawing = False
last_point = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)
    # Process the frame to detect hands
    results = hands.process(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger (landmark 8)
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            h, w, _ = frame.shape
            x_index, y_index = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            x_thumb, y_thumb = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Debugging: Draw circles at the thumb and index finger positions
            cv2.circle(frame, (x_index, y_index), 10, (0, 255, 0), -1)  # Index Finger (Green)
            cv2.circle(frame, (x_thumb, y_thumb), 10, (0, 0, 255), -1)  # Thumb (Red)

            # Print the distance for debugging
            distance = np.sqrt((x_index - x_thumb)**2 + (y_index - y_thumb)**2)
            print(f"Distance: {distance} pixels")  # This will print in the terminal

            # Check if the index and thumb tips are close enough to toggle drawing mode
            if distance < 50:  # 50 pixels threshold to toggle drawing
                drawing = not drawing
                last_point = None  # Reset drawing mode when toggling

            # Draw on the canvas if drawing mode is enabled
            if drawing:
                if last_point is not None:
                    cv2.line(canvas, last_point, (x_index, y_index), (0, 255, 0), 5)  # Green line
                last_point = (x_index, y_index)

    # Combine the video frame with the canvas
    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Display the frame with canvas
    cv2.imshow("Combined", combined)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the display window
cap.release()
cv2.destroyAllWindows()

Distance: 26.1725046566048 pixels
Distance: 8.48528137423857 pixels
Distance: 7.280109889280518 pixels
Distance: 23.853720883753127 pixels
Distance: 35.014282800023196 pixels
Distance: 39.319206502675 pixels
Distance: 30.066592756745816 pixels
Distance: 24.698178070456937 pixels
Distance: 32.202484376209235 pixels
Distance: 42.5440947723653 pixels
Distance: 18.384776310850235 pixels
Distance: 10.198039027185569 pixels
Distance: 22.02271554554524 pixels
Distance: 21.540659228538015 pixels
Distance: 26.570660511172846 pixels


In [25]:
# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Create a canvas
canvas = np.zeros((480, 640, 3), dtype="uint8")  # Black canvas

# Start video capture
cap = cv2.VideoCapture(0)

# Check if the webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Set webcam resolution to 640x480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Flag to track whether drawing is happening
drawing = False
last_point = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Process the frame to detect hands
    results = hands.process(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger (landmark 8) and thumb (landmark 4)
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            h, w, _ = frame.shape
            x_index, y_index = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            x_thumb, y_thumb = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Debugging: Draw circles at the thumb and index finger positions
            cv2.circle(frame, (x_index, y_index), 10, (0, 255, 0), -1)  # Index Finger (Green)
            cv2.circle(frame, (x_thumb, y_thumb), 10, (0, 0, 255), -1)  # Thumb (Red)

            # Print the distance for debugging
            distance = np.sqrt((x_index - x_thumb)**2 + (y_index - y_thumb)**2)
            print(f"Distance: {distance} pixels")  # This will print in the terminal

            # Check if the index and thumb tips are close enough to toggle drawing mode
            if distance < 30:  # 30 pixels threshold to toggle drawing
                if not drawing:  # Only toggle once
                    drawing = not drawing
                    print("Drawing mode activated" if drawing else "Drawing mode deactivated")
                    last_point = None  # Reset drawing mode when toggling

            # Draw on the canvas if drawing mode is enabled
            if drawing:
                if last_point is not None:
                    cv2.line(canvas, last_point, (x_index, y_index), (0, 255, 0), 5)  # Green line
                    print(f"Drawing line from {last_point} to {(x_index, y_index)}")  # Debugging line drawing
                last_point = (x_index, y_index)

    # Combine the video frame with the canvas
    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Display the frame with canvas
    cv2.imshow("Combined", combined)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the display window
cap.release()
cv2.destroyAllWindows()


Distance: 16.1245154965971 pixels
Drawing mode activated
Distance: 51.62363799656123 pixels
Drawing line from (208, 412) to (195, 397)
Distance: 49.39635614091387 pixels
Drawing line from (195, 397) to (188, 377)
Distance: 41.23105625617661 pixels
Drawing line from (188, 377) to (250, 349)
Distance: 22.561028345356956 pixels
Drawing line from (250, 349) to (233, 356)
Distance: 54.70831746635972 pixels
Drawing line from (233, 356) to (169, 167)
Distance: 47.29693436154187 pixels
Drawing line from (169, 167) to (161, 185)
Distance: 37.12142238654117 pixels
Drawing line from (161, 185) to (158, 197)
Distance: 39.6232255123179 pixels
Drawing line from (158, 197) to (154, 207)
Distance: 29.832867780352597 pixels
Drawing line from (154, 207) to (147, 213)
Distance: 12.083045973594572 pixels
Drawing line from (147, 213) to (142, 225)
Distance: 17.88854381999832 pixels
Drawing line from (142, 225) to (139, 240)
Distance: 18.788294228055936 pixels
Drawing line from (139, 240) to (138, 258)
Dist

In [27]:
# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Create a canvas
canvas = np.zeros((480, 640, 3), dtype="uint8")  # Black canvas

# Start video capture
cap = cv2.VideoCapture(0)

# Check if the webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Set webcam resolution to 640x480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Flag to track whether drawing is happening
drawing = False
last_point = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Process the frame to detect hands
    results = hands.process(frame)

    # Clear the canvas for the next frame
    canvas = np.zeros_like(canvas)  # Reset canvas to black each frame

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger (landmark 8) and thumb (landmark 4)
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            h, w, _ = frame.shape
            x_index, y_index = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            x_thumb, y_thumb = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Debugging: Draw circles at the thumb and index finger positions
            cv2.circle(frame, (x_index, y_index), 10, (0, 255, 0), -1)  # Index Finger (Green)
            cv2.circle(frame, (x_thumb, y_thumb), 10, (0, 0, 255), -1)  # Thumb (Red)

            # Print the distance for debugging
            distance = np.sqrt((x_index - x_thumb)**2 + (y_index - y_thumb)**2)
            print(f"Distance: {distance} pixels")  # This will print in the terminal

            # Check if the index and thumb tips are close enough to toggle drawing mode
            if distance < 30:  # 30 pixels threshold to toggle drawing
                if not drawing:  # Only toggle once
                    drawing = not drawing
                    print("Drawing mode activated" if drawing else "Drawing mode deactivated")
                    last_point = None  # Reset drawing mode when toggling

            # Draw on the canvas if drawing mode is enabled
            if drawing:
                if last_point is not None:
                    cv2.line(canvas, last_point, (x_index, y_index), (0, 0, 255), 5)  # Red line
                    print(f"Drawing line from {last_point} to {(x_index, y_index)}")  # Debugging line drawing
                last_point = (x_index, y_index)

    # Combine the video frame with the canvas
    combined = cv2.addWeighted(frame, 0.7, canvas, 0.3, 0)

    # Display the frame with canvas
    cv2.imshow("Combined", combined)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the display window
cap.release()
cv2.destroyAllWindows()


Distance: 67.26812023536856 pixels
Distance: 10.04987562112089 pixels
Drawing mode activated
Distance: 1.0 pixels
Drawing line from (402, 413) to (398, 396)
Distance: 60.67124524847005 pixels
Drawing line from (398, 396) to (366, 373)
Distance: 63.56886030125127 pixels
Drawing line from (366, 373) to (377, 378)
Distance: 19.1049731745428 pixels
Drawing line from (377, 378) to (409, 387)
Distance: 24.839484696748443 pixels
Drawing line from (409, 387) to (409, 392)
Distance: 32.802438933713454 pixels
Drawing line from (409, 392) to (415, 396)
Distance: 51.78802950489621 pixels
Drawing line from (415, 396) to (281, 333)
Distance: 27.784887978899608 pixels
Drawing line from (281, 333) to (276, 329)
Distance: 25.80697580112788 pixels
Drawing line from (276, 329) to (268, 332)
Distance: 6.708203932499369 pixels
Drawing line from (268, 332) to (255, 337)
Distance: 17.11724276862369 pixels
Drawing line from (255, 337) to (235, 131)
Distance: 13.038404810405298 pixels
Drawing line from (235, 1